In [1]:
# required Python imports
import numpy as np
import matplotlib.pyplot as plt
# from matplotlib import cm
from types import SimpleNamespace

import ftir_funct as f
np.set_printoptions(suppress=True)

module FTIR v.2024.2.22 imported


In [2]:
f.explore_Euler_space(step=1)

array([[  0. ,   0.1,   0. ],
       [  0. ,   0.1,   1. ],
       [  0. ,   0.1,   2. ],
       ...,
       [ 90. ,  88.1, 178. ],
       [ 90. ,  88.1, 179. ],
       [ 90. ,  88.1, 180. ]])

## Step 1: Create the database

Questions to be investigated:

- How many measurements are required to obtain the orientation with an error of, say, ±1 degree?
- What range of azimuths should we consider? 90, 180, 360? Does it matter?
- Are there special cases?
- Any others?

In [3]:
# STEP 1.1: generate Euler angles and store in the database
database = SimpleNamespace(euler=f.explore_Euler_space(step=30))
database.euler.shape

(84, 3)

In [4]:
# database.euler

In [5]:
# STEP 1.2: Generate a mesh of values defining the reference transmissión envelope
polar, azimuths = f.regular_S2_grid(n_squared=500)
T = f.Tvalues(trans=(90, 50, 20), azimuth=azimuths, polar=polar)
x, y, z = f.sph2cart(T, azimuths, polar)

In [6]:
# example consider the full range of azimuths and 16 measures
np.arange(0, 360, 45/2)

array([  0. ,  22.5,  45. ,  67.5,  90. , 112.5, 135. , 157.5, 180. ,
       202.5, 225. , 247.5, 270. , 292.5, 315. , 337.5])

In [7]:
# example consider the a 180 degrees range and 16 measures
np.arange(0, 180, 45/4)

array([  0.  ,  11.25,  22.5 ,  33.75,  45.  ,  56.25,  67.5 ,  78.75,
        90.  , 101.25, 112.5 , 123.75, 135.  , 146.25, 157.5 , 168.75])

In [8]:
# STEP 1.3: Generate 16 measures at different azimuth angles
angles = np.arange(0, 360, 45/2)

# initialize variables
T_vals = []
azi_vals = []

for euler in database.euler:
    # rotate
    x2, y2, z2 = f.rotate(coordinates=(x, y, z), euler_ang=euler)

    # extract XY intersection
    xy_vectors = f.extract_XY_section_fast2(x2, y2, z2)

    # get the indexes of specific angles
    indexes = f.find_nearest(xy_vectors['angles'], angles)

    # append values
    T_vals.append(xy_vectors.loc[indexes, ['T']].T.values.tolist()[0])
    azi_vals.append(xy_vectors.loc[indexes, ['angles']].T.values.tolist()[0])

# store values in database
database.T_values = np.array(T_vals)
database.azimuths = np.array(azi_vals)

In [9]:
print(database.euler.shape)
print(database.T_values.shape)
print(database.azimuths.shape)

(84, 3)
(84, 16)
(84, 16)


In [10]:
print('Euler angles: ', database.euler[-1])
print('T values: ', np.around(database.T_values[-1], 1))
print('Azimuths: ', np.around(database.azimuths[-1], 1))

Euler angles:  [ 90.   60.1 180. ]
T values:  [50.  48.2 43.7 39.3 37.4 39.3 43.7 48.2 50.  48.2 43.7 39.2 37.4 39.2
 43.7 48.2]
Azimuths:  [  0.1  22.4  44.8  67.3  90.2 112.7 135.2 157.6 179.9 202.4 225.  247.5
 270.  292.5 315.  337.6]


In [11]:
# for i in range(45):
#     print(np.around(database.azimuths[i], 1), '; ', np.around(database.euler[i], 0))

In [12]:
for index, orientation in enumerate(database.euler):
    print('Real:', np.around(orientation, 0))

    measures = np.column_stack((database.T_values[index],
                                database.azimuths[index],
                                np.full_like(database.azimuths[index], 90)))

    f.find_orientation(measurements=measures,
                       params=(90, 50, 20),
                       num_guesses=20)
    print('----------------------')

Real: [0. 0. 0.]
Calculated orientation: [ 62.   0. 118.]
----------------------
Real: [ 0.  0. 30.]
Calculated orientation: [ 48.   0. 162.]
----------------------
Real: [ 0.  0. 60.]
Calculated orientation: [ 73.   0. 167.]
----------------------
Real: [ 0.  0. 90.]
Calculated orientation: [51.  0. 39.]
----------------------
Real: [  0.   0. 120.]
Calculated orientation: [51.  0. 69.]
----------------------
Real: [  0.   0. 150.]
Calculated orientation: [56.  0. 94.]
----------------------
Real: [  0.   0. 180.]
Calculated orientation: [ 49.   0. 131.]
----------------------
Real: [ 0. 30.  0.]
Calculated orientation: [  0.  30. 180.]
----------------------
Real: [ 0. 30. 30.]
Calculated orientation: [ 1. 30. 29.]
----------------------
Real: [ 0. 30. 60.]
Calculated orientation: [ 1. 30. 59.]
----------------------
Real: [ 0. 30. 90.]
Calculated orientation: [ 0. 30. 90.]
----------------------
Real: [  0.  30. 120.]
Calculated orientation: [  1.  30. 119.]
----------------------
R

In [13]:
for index, orientation in enumerate(database.euler):
    print('Real:', np.around(orientation, 0))

    measures = np.column_stack((database.T_values[index],
                                database.azimuths[index],
                                np.full_like(database.azimuths[index], 90)))

    f.find_orientation_diffevol(measurements=measures, params=(90, 50, 20))
    print('----------------------')

Real: [0. 0. 0.]
Calculated Orientation: [ 51.   0. 129.]
----------------------
Real: [ 0.  0. 30.]
Calculated Orientation: [ 49.   0. 161.]
----------------------
Real: [ 0.  0. 60.]
Calculated Orientation: [53.  0.  7.]
----------------------
Real: [ 0.  0. 90.]
Calculated Orientation: [51.  0. 39.]
----------------------
Real: [  0.   0. 120.]
Calculated Orientation: [53.  0. 67.]
----------------------
Real: [  0.   0. 150.]
Calculated Orientation: [54.  0. 96.]
----------------------
Real: [  0.   0. 180.]
Calculated Orientation: [ 51.   0. 129.]
----------------------
Real: [ 0. 30.  0.]
Calculated Orientation: [  0.  30. 180.]
----------------------
Real: [ 0. 30. 30.]
Calculated Orientation: [ 1. 30. 29.]
----------------------
Real: [ 0. 30. 60.]
Calculated Orientation: [ 1. 30. 59.]
----------------------
Real: [ 0. 30. 90.]
Calculated Orientation: [ 0. 30. 90.]
----------------------
Real: [  0.  30. 120.]
Calculated Orientation: [  1.  30. 119.]
----------------------
Real

TODO:
- Carry out a test comparing the different methods at the same time (time, accuracy, etc.).
- See if there are any special situations, e.g. when Phi=0 is one case.
- The question of range of azimuths and number of points vs. accuracy.
- See how to deal with equivalent orientations, e.g. [60, 0, 0, 0] == [30, 0, 30] and so on.
